In [1]:
import functools
import re
import typing
import jax
import jax.numpy as jnp
from jax.scipy.spatial.transform import Rotation
from jax.scipy.spatial.transform import Rotation, Slerp
import jax.numpy as jnp


In [2]:
#adapted from https://stackoverflow.com/questions/74519927/best-way-to-rotate-and-translate-a-set-of-points-in-python

def rotate_and_translate(points: jnp.ndarray
                         , center_point: jnp.ndarray, rotation_vector: jnp.ndarray,
                         translation_vector: jnp.ndarray) -> jnp.ndarray:
    # rotation_matrix = Rotation.from_rotvec(rotation_vector).as_matrix()
    rotation_matrix =Rotation.from_euler('xyz', rotation_vector, degrees=False).as_matrix()
    return (points - center_point) @ rotation_matrix.T + center_point + translation_vector



# rotateMatrix = Rotation.from_rotvec(jnp.array([ jnp.pi,0.0, 0.0])).as_matrix()
points=jnp.array([[0.0, 0.0, 0.0], [0.0, 1.0, 0.0], [0.0, -1.0, 0.0]])
center_point=jnp.array([0.0, 0.0, 0.0])
# newxy = (points-center_point) @ rotateMatrix.T + center_point

newxy=rotate_and_translate(points, center_point, jnp.array([ jnp.pi,0.0, 0.0]), jnp.array([2.0, 2.0, 2.0]))
newxy.round(2)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


Array([[2., 2., 2.],
       [2., 1., 2.],
       [2., 3., 2.]], dtype=float32)

In [3]:
weights=jnp.array([1.0, 1.0, 1.0,2.0, 2.0, 2.0,3.0, 3.0, 3.0])
weights[0:3]
weights[3:6]

Array([2., 2., 2.], dtype=float32)

In [4]:
import functools
import re
import typing
import jax
import jax.numpy as jnp
from jax.scipy.spatial.transform import Rotation
import chex
from typing import Callable, Sequence, Tuple, Union
from dm_pix._src import augment
import numpy as np
import SimpleITK as sitk


def rotate_and_translate(points: jnp.ndarray
                         , center_point: jnp.ndarray, rotation_vector: jnp.ndarray,
                         translation_vector: jnp.ndarray) -> jnp.ndarray:
    # rotation_matrix = Rotation.from_rotvec(rotation_vector).as_matrix()

    rotation_matrix =Rotation.from_euler('xyz', rotation_vector, degrees=True).as_matrix()
    return (points - center_point) @ rotation_matrix.T + center_point + translation_vector


def get_fiducial_loos(weights,from_landmarsk,to_landmarks,image_shape):
    """
    first entries in in weights are :
        0-3 first rotation vector   
        3-6 translation vector   
    so we interpret the weights as input for transormations rotation;translation
    we apply this transformation to the fiducial points of moving image and 
    calculate the square distance between transformed fiducial points and fiducial points on fixed image           
    """
    center_point=(jnp.asarray(image_shape) - 1.) / 2
    res=rotate_and_translate(to_landmarks, center_point, weights[0:3],weights[3:6])
    #calculate the square distance between transformed fiducial points and fiducial points on fixed image 
    return jnp.sum(((from_landmarsk-res)**2).flatten())
    

# def affine_transform(
#     image: chex.Array,
#     matrix: chex.Array,
#     *,
#     offset: Union[chex.Array, chex.Numeric] = 0.,
#     order: int = 1,
#     mode: str = "constant",
#     cval: float = 0.0,
# ) -> chex.Array:
#   """Applies an affine transformation given by matrix.

#   """

#   meshgrid = jnp.meshgrid(*[jnp.arange(size) for size in image.shape],
#                           indexing="ij")
#   indices = jnp.concatenate(
#       [jnp.expand_dims(x, axis=-1) for x in meshgrid], axis=-1)

#   zz, yy, xx = meshgrid
#   z_center, y_center,x_center= (jnp.asarray(image.shape) - 1.) / 2.
#   indices = jnp.array([xx - x_center, yy - y_center, zz - z_center])

#   coordinates = jnp.tensordot(matrix, indices, axes=((1), (0)))

#   interpolate_function = augment._get_interpolate_function(
#       mode=mode,
#       order=order,
#       cval=cval,
#   )
#   return interpolate_function(image, coordinates)


    
def transform_image(image,weights):
    """
    first entries in in weights are :
    0-3 first rotation vector   
    3-6 translation vector   
    so we interpret the weights as input for transormations rotation;translation;rotation
    """      
    quaternion=np.array(Rotation.from_euler('xyz', weights[0:3], degrees=True).as_quat())
    # image_path="/root/data/pat_2/general_transform//study_0_ct_soft.nii.gz"
    # image_path_b="/workspaces/pilot_lymphoma/data/rotated.nii.gz"


    # Load the image
    image = sitk.GetImageFromArray(image)

    # Get the center of the image
    center = image.TransformContinuousIndexToPhysicalPoint([(index - 1) / 2.0 for index in image.GetSize()])

    # Create a VersorRigid3DTransform
    transform = sitk.VersorRigid3DTransform()
    transform.SetCenter(center)
    transform.SetRotation([float(quaternion[0]),float(quaternion[1]),float(quaternion[2]),float(quaternion[3])])

    # Resample the image
    resampler = sitk.ResampleImageFilter()
    resampler.SetOutputDirection(image.GetDirection())
    resampler.SetOutputOrigin(image.GetOrigin())
    resampler.SetOutputSpacing(image.GetSpacing())
    resampler.SetSize(image.GetSize())
    resampler.SetTransform(transform)

    resampled_image = resampler.Execute(image)
    res=sitk.GetArrayFromImage(resampled_image)
    # image=affine_transform(image,Rotation.from_euler('xyz', weights[0:3], degrees=True).inv().as_matrix())
    image=jnp.array(res)
    res=jax.image.scale_and_translate(image, image.shape,jnp.array([0,1,2]), jnp.array([1.0,1.0,1.0]), (weights[3:6]/20), "bicubic")
    return res





In [5]:
import SimpleITK as sitk
import jax.numpy as jnp


def resample_ct_to_suv(ct: sitk.Image, suv: sitk.Image) -> sitk.Image:
    """
    Resample a CT image to the same size as a SUV image
    """
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkBSpline)
    resampler.SetOutputSpacing(suv.GetSpacing())
    resampler.SetSize(suv.GetSize())
    resampler.SetOutputDirection(suv.GetDirection())
    resampler.SetOutputOrigin(suv.GetOrigin())
    ct= resampler.Execute(ct)
    
    ct_arr=sitk.GetArrayFromImage(ct)
    suv_arr=sitk.GetArrayFromImage(suv)
    
    res=jnp.stack([jnp.array(suv_arr),jnp.array(ct_arr)],axis=0)
    return res

def load_landmark_data(folder_path:str):
    """
    given path to folder with landmarks files and images after general registaration we load the data
    we want to first load the suv and ct images resample them to the same size and then load the landmarks
    we need to load separately study 0 and 1 
    the output should be in form of a dictionary with keys 'study_0','study_1','From`,`To`' where `From` and `To` are the landmarks
    all the data should be in form of jnp.arrays
    """
    ct_0=sitk.ReadImage(folder_path+'/study_0_ct_soft.nii.gz')
    suv_0=sitk.ReadImage(folder_path+'/study_0_SUVS.nii.gz')
    # Resample ct_0 to match ct_1
    arr_0 = resample_ct_to_suv(ct_0, suv_0)
            
    ct_1=sitk.ReadImage(folder_path+'/study_1_ct_soft.nii.gz')
    suv_1=sitk.ReadImage(folder_path+'/study_1_SUVS.nii.gz')    
    arr_1 = resample_ct_to_suv(ct_1, suv_1)

    return {'study_0':arr_0,'study_1':arr_1, 'From':jnp.load(folder_path+'/From.npy'),'To':jnp.load(folder_path+'/To.npy')}




folder_path='/root/data/pat_2/general_transform'
load_landmark_data(folder_path)

# cp /root/data/pat_2/To.npy /root/data/pat_2/general_transform/To.npy

{'study_0': Array([[[[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          [0., 0., 0., ..., 0., 0., 0.],
          ...,
          [0., 0., 0., ..., 0., 0., 0.],
  

In [6]:
import os
import jax
import jax.numpy as jnp

folder_path = "/root/data"
folder_names = [name for name in os.listdir(folder_path) if os.path.isdir(os.path.join(folder_path, name))]
folder_names=list(filter(lambda x: re.match(r'pat_\d+',x),folder_names))
folder_names=list(map(lambda x: f"{folder_path}/{x}/general_transform",folder_names))

curr_folder='/root/data/pat_2/general_transform'
data_dict=load_landmark_data(curr_folder)

# v_transform_image=jax.vmap(transform_image , in_axes = (0,None) )

def get_data_for_pretained_model(image_dat,landmark_data, rng):
    """
    given the image data and landmark data we return the data in the format that can be used for the pretrained model
    so we make a random transformation of the image and the landmarks and return the transformed image and landmarks 
    """
    #random weights for the transformation
    weights = jax.random.uniform(rng, shape=(6,), minval=0, maxval=360)
    # print(f"image_dat {image_dat.shape} landmark_data {landmark_data.shape} weights {weights.shape}")
    tansf_image=jnp.stack([transform_image(image_dat[0,:,:,:],weights),transform_image(image_dat[1,:,:,:],weights)   ])
    center_point=(jnp.asarray(image_dat.shape) - 1.) / 2
    center_point=center_point[0:3]
    # print(f" landmark_data {landmark_data.shape} center_point {center_point} weights {weights.shape}")
    transf_landmarsk=rotate_and_translate(landmark_data, center_point,weights[0:3],(weights[3:6]/10))


    return tansf_image,transf_landmarsk

# krowa transform the image save it and check if it tranforms correctly
 

random_seed = 42
rng = jax.random.PRNGKey(random_seed)
tansf_image,transf_landmarsk=get_data_for_pretained_model(data_dict['study_0'],data_dict['From'], rng)



image_dat (2, 425, 200, 200) landmark_data (5, 3) weights (6,)
 landmark_data (5, 3) center_point [  0.5 212.   99.5] weights (6,)


In [7]:
out_folder_path="/workspaces/pilot_lymphoma/data"
import SimpleITK as sitk
import numpy as np

# Assuming `transf_image` and `data_dict['study_0']` are JAX arrays
transf_image_np = np.array(tansf_image[0,:,:,:])
study_0_np = np.array(data_dict['study_0'][0,:,:,:])

# Create SimpleITK images
transf_image_sitk = sitk.GetImageFromArray(transf_image_np)
study_0_sitk = sitk.GetImageFromArray(study_0_np)

# Save as NIfTI files
sitk.WriteImage(transf_image_sitk, '/workspaces/pilot_lymphoma/data/transf_image.nii.gz')
sitk.WriteImage(study_0_sitk, '/workspaces/pilot_lymphoma/data/study_0.nii.gz')

In [9]:
def save_landmarks_for_slicer(points, file_path):

    # Create the header
    header = "# Markups fiducial file version = 4.6\n# CoordinateSystem = 0\n# columns = id,x,y,z,ow,ox,oy,oz,vis,sel,lock,label,desc,associatedNodeID\n"

    # Create the body
    body = "\n".join([f"{i},{x},{y},{z},0,0,0,1,1,1,0,F{i+1},," for i, (x, y, z) in enumerate(points)])

    # Combine header and body
    fcsv_content = header + body

    # Write to file
    with open(file_path, "w") as f:
        f.write(fcsv_content)
        
ct_0=sitk.ReadImage(folder_path+'/study_0_ct_soft.nii.gz')
From=jnp.load(folder_path+'/From.npy')
transf_landmarsk=rotate_and_translate(landmark_data, center_point,weights[0:3],(weights[3:6]/10))
 
save_landmarks_for_slicer(data_dict['From'], "/workspaces/pilot_lymphoma/data/From.fcsv")
save_landmarks_for_slicer(transf_landmarsk, "/workspaces/pilot_lymphoma/data/From_transformed.fcsv")


In [8]:
# r = jax.scipy.spatial.transform.Rotation.from_rotvec(jnp.array([jnp.pi,jnp.pi, jnp.pi])) ####### it is in radians
# quaternion=np.array(r.as_quat())
# image_path="/root/data/pat_2/general_transform//study_0_ct_soft.nii.gz"
# image_path_b="/workspaces/pilot_lymphoma/data/rotated.nii.gz"

# import SimpleITK as sitk

# # Load the image
# image = sitk.ReadImage(image_path)

# # Get the center of the image
# center = image.TransformContinuousIndexToPhysicalPoint([(index - 1) / 2.0 for index in image.GetSize()])

# # Create a VersorRigid3DTransform
# transform = sitk.VersorRigid3DTransform()
# transform.SetCenter(center)
# transform.SetRotation([float(quaternion[0]),float(quaternion[1]),float(quaternion[2]),float(quaternion[3])])

# # Resample the image
# resampler = sitk.ResampleImageFilter()
# resampler.SetOutputDirection(image.GetDirection())
# resampler.SetOutputOrigin(image.GetOrigin())
# resampler.SetOutputSpacing(image.GetSpacing())
# resampler.SetSize(image.GetSize())
# resampler.SetTransform(transform)

# resampled_image = resampler.Execute(image)

# # Save the resampled image
# sitk.WriteImage(resampled_image, image_path_b)
# # saver.save(rotated_image, image_path_b)